In [ ]:
#%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func 
from sqlalchemy import inspect 

In [ ]:
# create engine and inspector to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
insp = inspect(engine)

# list the tables 
print(insp.get_table_names())

List the columns in the measurement and station tables


In [ ]:
s_cols = insp.get_columns('station')
print("Columns in the station table ")
for col in s_cols:
    print(col['name'], col['type'])

print(" ")
m_cols = insp.get_columns('measurement')
print("Columns in the measurement table ")
for col in m_cols:
    print(col['name'], col['type'])

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement 
Station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
end_date_str = session.query(Measurement.date).order_by(Measurement.date.desc()).first()

The data are daily rainfall totals and some kind of temperature, possibly daily max.  
I will import data from all the stations, so the date will not be able to be used as an index.

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
end_date = date(*map(int, end_date_str[0].split('-')))

# Calculate the date one year from the last date in data set.
start_date = end_date - dt.timedelta(days=365)
print(f"start date = {start_date}, end date = {end_date}")

# Perform a query to retrieve the data and precipitation scores
start_date_str = start_date.strftime("%Y-%m-%d")
precip_df = pd.DataFrame(session.query(Measurement.date, Measurement.station, Measurement.prcp).filter(Measurement.date > start_date_str).all())

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date (and station number)  
precip_df = precip_df.sort_values(by=['date', 'station'])  

#convert the date string into datetime 
precip_df['date'] = pd.to_datetime(precip_df['date'])
precip_df.info()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
# I will plot out the mean from all the stations as a time series 
mean_precip = precip_df[['date','prcp']].groupby(by='date').mean()

fig,ax = plt.subplots(figsize=(15,6))
ax.bar(mean_precip.index,mean_precip['prcp'],width=1.0)
ax.set(xlabel="Date",
       ylabel="Precipitation (inches)",
       title="Daily Mean Precipitation")

# Major ticks every month.
fmt_month = mdates.MonthLocator(interval=1)
ax.xaxis.set_major_locator(fmt_month)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
fig.autofmt_xdate() 
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
cols = ['count', 'mean','std', 'min', '25%', '50%', '75%', 'max']
station_list = precip_df['station'].unique()
station_summary_df = pd.DataFrame(columns=cols, index=station_list)

for station in station_list:
    results = precip_df.loc[precip_df['station'] == station].describe()
    station_summary_df.loc[station] = results['prcp']

station_summary_df = station_summary_df.sort_values(by='count', ascending=False)
station_summary_df    

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
number_stations = session.query(Station.id).count()
print(f"Number of stations = {number_stations}")

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
number_days = session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).order_by(func.count(Measurement.tobs).desc()).all()

print("Total number of observations per station \nStation        Number of days")
for day in number_days:
    print(f"{day[0]}       {day[1]}")  

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
max_stn = number_days[0][0]
min_temp = session.query(func.min(Measurement.tobs)).filter(Measurement.station == max_stn).all()
max_temp = session.query(func.max(Measurement.tobs)).filter(Measurement.station == max_stn).all()
mean_temp = session.query(func.avg(Measurement.tobs)).filter(Measurement.station == max_stn).all()
print(f"Station id {max_stn}: min temp = {min_temp[0][0]:.2f}F, mean temp = {mean_temp[0][0]:.2f}F, max temp = {max_temp[0][0]:.2f}F")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
max_tobs = session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).order_by(func.count(Measurement.tobs).desc()).first()
print(f"Station {max_tobs[0]}, number of observations = {max_tobs[1]}")
tobs_df = pd.DataFrame(session.query(Measurement.date, Measurement.tobs).filter(Measurement.station == max_tobs[0], Measurement.date > start_date_str).all())

# Sort the dataframe by date, convert into datetime, and set as the index 
tobs_df = tobs_df.sort_values(by=['date'])  
tobs_df['date'] = pd.to_datetime(tobs_df['date'])
tobs_df = tobs_df.set_index('date')
tobs_df

In [ ]:
fig,ax = plt.subplots(figsize=(12,10))
ax.hist(tobs_df['tobs'],bins=12)
ax.set(xlabel="Temperature (F)",
       ylabel="Frequency",
       title=f"Daily max. temperature \n Station {max_tobs[0]}")
plt.show()

# Close session

In [ ]:
# Close Session
session.close()